In [3]:
import numpy as np

1. 对于元素为 $A_{ij}=5(i+2j-1)$ 的 $n\times n$ 矩阵，令 $x=[1,\,\dots\,,1]^T, \;b=Ax$. 使用2.1.1的程序（高斯消元法）计算双精度解 $x_c$. 找出 $Ax=b$ 的前向误差和误差放大因子的无穷范数，并于A的条件数进行比较: $(a)\;n=6,\quad(b)\;n=10$ .

In [4]:
pass

2. 对于 $A_{ij}=1/(|i-j|+1)$ 的矩阵解答1中的问题。

In [5]:
pass

3. 对于元素为 $A_{ij}=|i-j|+1$ 的 $n\times n$ 矩阵，解答1中的问题，其中 $n=100,\;200,\;300,\;400,\;500$. 并找出问题 $Ax=b$ 的5个误差放大因子，并和对应的条件数进行比较。


In [6]:
pass

4. 对于 $A_{ij}=\sqrt{(i-j)^2+n/10}$ 的矩阵解答3中的问题。

In [7]:
pass

5. n取多少时，问题1中的解没有正确的有效数字？

In [8]:
pass

6. 使用2.1.1的程序（高斯消元法）解答例2.13（见下）中的第2和第3中方法的双精度实现，并和课本中的理论解进行比较。
$$
\begin{aligned}
10^{-20}x_1+x_2&=1\\
x_1+2x_2&=4
\end{aligned}
$$

In [9]:
pass